In [2]:
%pwd

'/home/sarath_kumar/ImagoAI/notebook'

In [ ]:
# import os

# os.chdir("../")
# %pwd

'/home/sarath_kumar/ImagoAI'

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import optuna
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("data/TASK-ML-INTERN.csv")

# Log-transform the target variable to reduce skewness
df["vomitoxin_ppb"] = np.log1p(df["vomitoxin_ppb"])

# Identify and remove outliers using IQR
Q1 = df["vomitoxin_ppb"].quantile(0.25)
Q3 = df["vomitoxin_ppb"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df["vomitoxin_ppb"] >= lower_bound) & (df["vomitoxin_ppb"] <= upper_bound)]

# Normalize spectral features using MinMaxScaler
scaler = MinMaxScaler()
X = df.iloc[:, 1:-1].values  # Selecting spectral reflectance features
X_scaled = scaler.fit_transform(X)
y = df["vomitoxin_ppb"].values  # Transformed target

# Apply PCA to retain more variance
pca = PCA(n_components=150)  # Increase components to 100
X_pca = pca.fit_transform(X_scaled)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define Optuna objective function for hyperparameter tuning


In [5]:
def build_mlp():
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Train the MLP model
mlp_model = build_mlp()
mlp_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1)

# Evaluate the MLP model
test_loss, test_mae = mlp_model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-15 11:30:42.410053: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 39.5683 - mae: 6.1029 - val_loss: 21.5681 - val_mae: 4.2438
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 17.9464 - mae: 3.6865 - val_loss: 16.0637 - val_mae: 2.9674
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 11.8311 - mae: 2.4752 - val_loss: 9.1340 - val_mae: 2.5949
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 8.5634 - mae: 2.3814 - val_loss: 6.6096 - val_mae: 2.1716
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.6278 - mae: 1.8465 - val_loss: 4.9944 - val_mae: 1.8551
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.5662 - mae: 1.5367 - val_loss: 3.7471 - val_mae: 1.6216
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.2084 - mae: 1.1777 - val_loss: 2.9447 - val_mae: 1.3579
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.6775 - mae: 1.0025 - val_loss: 2.4466 - val_mae: 1.2747
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

In [ ]:
from tensorflow.keras import layers, regularizers
# Load dataset
df = pd.read_csv("data/TASK-ML-INTERN.csv")

# Log-transform the target variable to reduce skewness
df["vomitoxin_ppb"] = np.log1p(df["vomitoxin_ppb"])

# Identify and handle outliers using Winsorization
Q1 = df["vomitoxin_ppb"].quantile(0.25)
Q3 = df["vomitoxin_ppb"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df["vomitoxin_ppb"] = np.clip(df["vomitoxin_ppb"], lower_bound, upper_bound)

# Normalize spectral features using MinMaxScaler
scaler = MinMaxScaler()
X = df.iloc[:, 1:-1].values  # Selecting spectral reflectance features
X_scaled = scaler.fit_transform(X)
y = df["vomitoxin_ppb"].values  # Transformed target

# Apply PCA to retain optimal variance (Reduced to 100 components)
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_scaled)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define and train an optimized Multi-Layer Perceptron (MLP) model
def build_mlp():
    model = keras.Sequential([
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.Huber(), metrics=['mae'])
    return model

# Train the optimized MLP model
mlp_model = build_mlp()
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
mlp_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)

# Evaluate the optimized MLP model
y_test_pred = mlp_model.predict(X_test)
r2 = r2_score(y_test, y_test_pred)
test_loss, test_mae = mlp_model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")
print(f"Test R² Score: {r2}")


Epoch 1/100


/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 9.3734 - mae: 5.6301 - val_loss: 7.8450 - val_mae: 4.6806
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 7.2357 - mae: 4.2222 - val_loss: 5.5136 - val_mae: 2.9310
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.6971 - mae: 3.2551 - val_loss: 4.8060 - val_mae: 2.7202
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.0774 - mae: 3.1188 - val_loss: 4.2558 - val_mae: 2.5643
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.2109 - mae: 2.6126 - val_loss: 3.6499 - val_mae: 2.2581
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.8493 - mae: 2.5224 - val_loss: 3.1517 - val_mae: 1.9794
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.2108 - mae: 2.0749 - val_loss: 2.7696 - val_mae: 1.7649
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.9040 - mae: 1.9384 - val_loss: 2.6026 - val_mae: 1.6958
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.8051 -

In [11]:


# Define Optuna objective function for hyperparameter tuning
def objective(trial):
    model = keras.Sequential([
        layers.Dense(trial.suggest_int('units_l1', 128, 512), activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
        layers.Dropout(trial.suggest_float('dropout_l1', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l2', 64, 256), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(trial.suggest_float('dropout_l2', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l3', 32, 128), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1)
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=trial.suggest_float('lr', 5e-4, 1e-2, log=True))
    model.compile(optimizer=optimizer, loss=keras.losses.Huber(), metrics=['mae'])
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=trial.suggest_int('epochs', 50, 150), batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]), callbacks=[early_stopping], verbose=0)
    
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Train final model with best hyperparameters
best_params = study.best_params
final_model = keras.Sequential([
    layers.Dense(best_params['units_l1'], activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dropout(best_params['dropout_l1']),
    layers.Dense(best_params['units_l2'], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(best_params['dropout_l2']),
    layers.Dense(best_params['units_l3'], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

final_optimizer = keras.optimizers.Adam(learning_rate=best_params['lr'])
final_model.compile(optimizer=final_optimizer, loss=keras.losses.Huber(), metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping], verbose=1)

# Evaluate final model
y_test_pred = final_model.predict(X_test)
r2 = r2_score(y_test, y_test_pred)
test_loss, test_mae = final_model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")
print(f"Test R² Score: {r2}")

[I 2025-03-15 11:39:34,831] A new study created in memory with name: no-name-9adaa9c5-0df6-46da-8915-a0b74fab8403


/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


[I 2025-03-15 11:39:42,156] Trial 0 finished with value: 0.1805069062186041 and parameters: {'units_l1': 450, 'dropout_l1': 0.14175530034082626, 'units_l2': 146, 'dropout_l2': 0.3120105898920607, 'units_l3': 75, 'lr': 0.004808534492062374, 'epochs': 97, 'batch_size': 16}. Best is trial 0 with value: 0.1805069062186041.
/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


[I 2025-03-15 11:42:33,047] Trial 19 finished with value: 0.206728916096255 and parameters: {'units_l1': 434, 'dropout_l1': 0.4876251035516389, 'units_l2': 90, 'dropout_l2': 0.4286034984361798, 'units_l3': 67, 'lr': 0.003971823070207044, 'epochs': 64, 'batch_size': 32}. Best is trial 10 with value: 0.23417696469395666.


Epoch 1/57


/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 6.0975 - mae: 4.3896 - val_loss: 2.9034 - val_mae: 2.5912
Epoch 2/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.9022 - mae: 2.6215 - val_loss: 1.9864 - val_mae: 1.8047
Epoch 3/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.1292 - mae: 2.0057 - val_loss: 1.7263 - val_mae: 1.6822
Epoch 4/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.0003 - mae: 2.0077 - val_loss: 1.7658 - val_mae: 1.8540
Epoch 5/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.8304 - mae: 1.9134 - val_loss: 1.6587 - val_mae: 1.7932
Epoch 6/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.6422 - mae: 1.7619 - val_loss: 1.4795 - val_mae: 1.6029
Epoch 7/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.7642 - mae: 1.9375 - val_loss: 1.5397 - val_mae: 1.7312
Epoch 8/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.7991 - mae: 1.9983 - val_loss: 1.4571 - val_mae: 1.6239
Epoch 9/57
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.6311 - mae: 1.

In [12]:
y_test_pred = mlp_model.predict(X_test)
r2 = r2_score(y_test, y_test_pred)
test_loss, test_mae = mlp_model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")
print(f"Test R² Score: {r2}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.7437 - mae: 1.7751
Test MAE: 1.6613751649856567
Test R² Score: 0.1457977142744621


### **Model Performance Interpretation**

#### **1. Optimized MLP Performance**
- **Test MAE: 1.44** → The model's average prediction error is **1.44 ppb** in log-transformed DON concentration.
- **R² Score: 0.185** → The model explains **18.5%** of the variance in DON concentration, which is still low.

#### **2. Key Takeaways**
- The **R² score improved slightly** compared to previous runs, but it's still relatively low.
- **Possible reasons for limited performance:**
  - **High variance in DON concentration**: Even after log transformation, extreme values may still impact predictions.
  - **Spectral Data Complexity**: Some important spectral bands might have been lost in PCA.
  - **Neural Network Limitations**: The model may struggle with **non-linear relationships** in the data.


#### **4. Next Steps**
- **Increase PCA components further** (e.g., **180-200**) to retain more variance.
- **Try ensemble methods (XGBoost + MLP stacking)** for better performance.
- **Experiment with different architectures (Residual Connections, Transformers)**.


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

# Train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)
y_xgb_pred = xgb_model.predict(X_test)

# Evaluate XGBoost model
xgb_r2 = r2_score(y_test, y_xgb_pred)
xgb_mae = mean_absolute_error(y_test, y_xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, y_xgb_pred))
print(f"XGBoost Test MAE: {xgb_mae}")
print(f"XGBoost Test RMSE: {xgb_rmse}")
print(f"XGBoost Test R² Score: {xgb_r2}")

# Define Optuna objective function for hyperparameter tuning
def objective(trial):
    model = keras.Sequential([
        layers.Dense(trial.suggest_int('units_l1', 128, 512), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
        layers.Dropout(trial.suggest_float('dropout_l1', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l2', 64, 256), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(trial.suggest_float('dropout_l2', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l3', 32, 128), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1)
    ])
    
    optimizer = keras.optimizers.AdamW(learning_rate=trial.suggest_float('lr', 5e-4, 1e-2, log=True))
    model.compile(optimizer=optimizer, loss=keras.losses.Huber(), metrics=['mae'])
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=trial.suggest_int('epochs', 50, 150), batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]), callbacks=[early_stopping], verbose=0)
    
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Train final model with best hyperparameters
best_params = study.best_params
final_model = keras.Sequential([
    layers.Dense(best_params['units_l1'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dropout(best_params['dropout_l1']),
    layers.Dense(best_params['units_l2'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(best_params['dropout_l2']),
    layers.Dense(best_params['units_l3'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

final_optimizer = keras.optimizers.AdamW(learning_rate=best_params['lr'])
final_model.compile(optimizer=final_optimizer, loss=keras.losses.Huber(), metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping], verbose=1)

# Evaluate final model
y_test_pred = final_model.predict(X_test)
r2 = r2_score(y_test, y_test_pred)
test_loss, test_mae = final_model.evaluate(X_test, y_test)
print(f"Optimized MLP Test MAE: {test_mae}")
print(f"Optimized MLP Test R² Score: {r2}")


[I 2025-03-15 11:46:39,501] A new study created in memory with name: no-name-c59e5eaf-8efc-42db-b8da-1c3a1cdc09c0


XGBoost Test MAE: 1.66173920226296
XGBoost Test RMSE: 2.1803631071727163
XGBoost Test R² Score: 0.23307900383496583


/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


[I 2025-03-15 11:46:50,177] Trial 0 finished with value: 0.18583118154963651 and parameters: {'units_l1': 355, 'dropout_l1': 0.1456545991389313, 'units_l2': 149, 'dropout_l2': 0.396992357513159, 'units_l3': 114, 'lr': 0.004182670406044688, 'epochs': 69, 'batch_size': 16}. Best is trial 0 with value: 0.18583118154963651.
/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


[I 2025-03-15 11:49:31,592] Trial 19 finished with value: 0.15572802486161075 and parameters: {'units_l1': 279, 'dropout_l1': 0.16577150829804949, 'units_l2': 68, 'dropout_l2': 0.36941886138764285, 'units_l3': 71, 'lr': 0.003154274769852997, 'epochs': 99, 'batch_size': 32}. Best is trial 18 with value: 0.23037839624302714.


Epoch 1/97


/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 7.0307 - mae: 4.3585 - val_loss: 3.5482 - val_mae: 2.4406
Epoch 2/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.4433 - mae: 2.5438 - val_loss: 2.3373 - val_mae: 1.8333
Epoch 3/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 2.2031 - mae: 1.7553 - val_loss: 1.9632 - val_mae: 1.6801
Epoch 4/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.0561 - mae: 1.8165 - val_loss: 1.8835 - val_mae: 1.7222
Epoch 5/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.9036 - mae: 1.7523 - val_loss: 1.7074 - val_mae: 1.6270
Epoch 6/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.8358 - mae: 1.7870 - val_loss: 1.7763 - val_mae: 1.7850
Epoch 7/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.7566 - mae: 1.7724 - val_loss: 1.8328 - val_mae: 1.8595
Epoch 8/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.7432 - mae: 1.7956 - val_loss: 1.6133 - val_mae: 1.6853
Epoch 9/97
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.5832 - mae: 1.64

### **Optimized MLP Model Interpretation**
**Test MAE: 1.64** → The average absolute error is **1.64 ppb** in the log-transformed DON concentration.  
**Test R² Score: 0.198** → The model explains **19.8%** of the variance in DON concentration.  

### **What This Means**
1. **Slight Improvement**  
   - Compared to previous results, the **R² score increased slightly**, indicating that the model is learning **better patterns** in the data.  
   - However, a score of **~0.20 R²** still means the model isn't capturing most of the variance.  

2. **Possible Issues**
   - **PCA (200 components)**: Might still be **removing too much information**.
   - **Deep Learning Limitations**: MLP might not be the best choice for this **high-dimensional structured data**.
   - **Non-linearity in Data**: XGBoost and stacking might still outperform MLP due to tree-based models handling **complex relationships** better.


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# Load dataset
df = pd.read_csv("data/TASK-ML-INTERN.csv")

# Log-transform the target variable to reduce skewness
df["vomitoxin_ppb"] = np.log1p(df["vomitoxin_ppb"])

# Identify and handle outliers using Winsorization
Q1 = df["vomitoxin_ppb"].quantile(0.25)
Q3 = df["vomitoxin_ppb"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df["vomitoxin_ppb"] = np.clip(df["vomitoxin_ppb"], lower_bound, upper_bound)

# Normalize spectral features using MinMaxScaler
scaler = MinMaxScaler()
X = df.iloc[:, 1:-1].values  # Selecting spectral reflectance features
X_scaled = scaler.fit_transform(X)
y = df["vomitoxin_ppb"].values  # Transformed target

# Apply PCA to retain optimal variance (Increased to 200 components)
pca = PCA(n_components=200)
X_pca = pca.fit_transform(X_scaled)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)
y_xgb_pred = xgb_model.predict(X_test)

# Evaluate XGBoost model
xgb_r2 = r2_score(y_test, y_xgb_pred)
xgb_mae = mean_absolute_error(y_test, y_xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, y_xgb_pred))
print(f"XGBoost Test MAE: {xgb_mae}")
print(f"XGBoost Test RMSE: {xgb_rmse}")
print(f"XGBoost Test R² Score: {xgb_r2}")

# Define Optuna objective function for hyperparameter tuning
def objective(trial):
    model = keras.Sequential([
        layers.Dense(trial.suggest_int('units_l1', 128, 512), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
        layers.Dropout(trial.suggest_float('dropout_l1', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l2', 64, 256), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(trial.suggest_float('dropout_l2', 0.1, 0.5)),
        layers.Dense(trial.suggest_int('units_l3', 32, 128), activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1)
    ])
    
    optimizer = keras.optimizers.AdamW(learning_rate=trial.suggest_float('lr', 5e-4, 1e-2, log=True))
    model.compile(optimizer=optimizer, loss=keras.losses.Huber(), metrics=['mae'])
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=trial.suggest_int('epochs', 50, 150), batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]), callbacks=[early_stopping], verbose=0)
    
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Train final model with best hyperparameters
best_params = study.best_params
final_mlp_model = keras.Sequential([
    layers.Dense(best_params['units_l1'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dropout(best_params['dropout_l1']),
    layers.Dense(best_params['units_l2'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(best_params['dropout_l2']),
    layers.Dense(best_params['units_l3'], activation='leaky_relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

final_optimizer = keras.optimizers.AdamW(learning_rate=best_params['lr'])
final_mlp_model.compile(optimizer=final_optimizer, loss=keras.losses.Huber(), metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
final_mlp_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping], verbose=1)

# Stacking Ensemble (XGBoost + MLP)
stacking_model = StackingRegressor(
    estimators=[('xgb', xgb_model)],
    final_estimator=Ridge()
)
stacking_model.fit(X_train, y_train)
y_stack_pred = stacking_model.predict(X_test)
stack_r2 = r2_score(y_test, y_stack_pred)
stack_mae = mean_absolute_error(y_test, y_stack_pred)
stack_rmse = np.sqrt(mean_squared_error(y_test, y_stack_pred))
print(f"Stacking Model Test MAE: {stack_mae}")
print(f"Stacking Model Test RMSE: {stack_rmse}")
print(f"Stacking Model Test R² Score: {stack_r2}")

In [ ]:

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor

from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import Ridge


# Define multiple ML models
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
lgb_model = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
adaboost_model = AdaBoostRegressor(n_estimators=200, learning_rate=0.05 ,random_state=42)
gbr_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)

# Train models
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)
adaboost_model.fit(X_train, y_train)
gbr_model.fit(X_train, y_train)

# Create a Voting Regressor (Boosting Ensemble)
voting_model = VotingRegressor(estimators=[('xgb', xgb_model), ('lgb', lgb_model), ('cat', adaboost_model), ('gbr', gbr_model)])
voting_model.fit(X_train, y_train)



In [24]:
# Evaluate Voting Regressor
y_voting_pred = voting_model.predict(X_test)
voting_r2 = r2_score(y_test, y_voting_pred)
voting_mae = mean_absolute_error(y_test, y_voting_pred)
voting_rmse = np.sqrt(mean_squared_error(y_test, y_voting_pred))
print(f"Voting Regressor Test MAE: {voting_mae}")
print(f"Voting Regressor Test RMSE: {voting_rmse}")
print(f"Voting Regressor Test R² Score: {voting_r2}")

/home/sarath_kumar/ImagoAI/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Voting Regressor Test MAE: 1.674456453617505
Voting Regressor Test RMSE: 2.1753289307744588
Voting Regressor Test R² Score: 0.23661635823583316


### **Voting Regressor Model Performance Interpretation**
✅ **Test MAE: 1.67** → The average absolute error is **1.67 ppb** in log-transformed DON concentration.  
✅ **Test RMSE: 2.18** → The model’s root mean squared error is **2.18 ppb**, showing how much it deviates from actual values.  
✅ **Test R² Score: 0.237** → The model explains **23.7% of the variance**, which is an **improvement** but still **not ideal**.


### **What This Means**
1. **Better than MLP (R² ≈ 0.198)**
   - The **Voting Regressor (stacking multiple models)** improved the variance explanation.
   - However, an R² of **~0.24** is still low, meaning **more than 75% of the variance is unexplained**.

2. **Possible Issues**
   - **Spectral Features Might Need Different Processing** → PCA (200 components) might **remove useful information**.  
   - **Boosting Models Need Further Tuning** → XGBoost, LightGBM, and Adaboost may need **hyperparameter tuning**.

